In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pickle

import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.reinforce import reinforce_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import py_tf_eager_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

import random
import os
import subprocess
from itertools import permutations 
from itertools import combinations 
import sys
from random import randint
from datetime import datetime

from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy



import abc
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

from datetime import datetime

In [ ]:
# Restart monitor
kill_pcm()
start_pcm()

In [ ]:
# Values

stored = 0
num_execution_core = 6
NUM_CPUS = 24
performance_counter_keys = ['MEM_INST_RETIRED.ALL_LOADS', 'L1D.REPLACEMENT', 'MEM_LOAD_RETIRED.L2_MISS', 'MEM_LOAD_RETIRED.L3_MISS', 'IPS', 'L1_MISS', 'L2_MISS', 'L3_MISS']

In [ ]:
# Get HPC and pre-process HPC

def profiling_cpu(df_dict):
    cpu_counters = []
    for df in performance_counter_keys:
        #rint(df_dict[df].iloc[0])
        for df_i in df_dict[df].iloc[0]:
            cpu_counters.append(df_i)

    #print(cpu_counters)
    cpu_counters_counter = []

    for counter in range(int(len(cpu_counters)/NUM_CPUS)):
        this_cpu_counter = []
        for cpu in range(NUM_CPUS):
            this_cpu_counter.append(cpu_counters[cpu+NUM_CPUS*counter])
        cpu_counters_counter.append(this_cpu_counter)
    
    # NORMALIZE
    max_list = []
    for i in range(len(cpu_counters_counter)):
        max_c = 0
        for j in range(NUM_CPUS):
            if cpu_counters_counter[i][j] > max_c:
                max_c = cpu_counters_counter[i][j]
        max_list.append(max_c)

    
    cpu_counters_normalized = []
    for i in range(len(cpu_counters_counter)):
        cpu_counters_normalized_this = []
        for j in range(NUM_CPUS):
            cpu_counters_normalized_this.append(cpu_counters_counter[i][j]/max_list[i]/2)
        cpu_counters_normalized.append(cpu_counters_normalized_this)
    return cpu_counters_normalized

In [ ]:
# Update HPC

def update_observation():
    global stored
    while(True):
        try:
            df_dict = get_microarchitecture_snapshot_2()
            if df_dict['L3_MISS'].iloc[-1][0] != stored:
                #print(df_dict['L3_MISS'].iloc[-1][0])
                stored = df_dict['L3_MISS'].iloc[-1][0]
                #profiling_input(df_dict.iloc[-1])
                break
            else:
                print("no update collected")
                time.sleep(1)
        except Exception as e:
            print("no data collected", e)
            time.sleep(1)
    global num_execution_core
    observation = profiling_cpu(df_dict)
    observation.append([num_execution_core/NUM_CPUS]*24)
    return observation

In [ ]:
hardware_counters = update_observation()

In [ ]:
# Update deployment

def get_current_execution_state():
    return num_execution_core

def update_current_execution_state(op):
    global num_execution_core
    if op == 1:
        if num_execution_core >= 20:
            return -1
        else:
            num_execution_core += 1
    else:
        if num_execution_core <= 4:
            return -1
        else:
            num_execution_core -=1
    return 0

In [ ]:
def update_deployment():
    nflambda = NFlambda(start = True)
    
    nflambda.parameter = Parameter()
    nflambda.parameter.d["state_size"] = 1000000
    nflambda.parameter.d["data_state"] = 4000
    nflambda.parameter.d["control_state"] = 100
    nflambda.parameter.d["control_frequency"] = 5
    nflambda.parameter.d["control_complexity"] = 500 # Tested: 500
    nflambda.configure_parameter(nflambda.parameter)
    for i in nfunctor_graph.state_runtime_table.data:
        state_id = nfunctor_graph.state_runtime_table.data.index(i)
        nflambda.send_command(
                command_state.with_argument(["create", str(0), str(state_id)]))
        
        
    decomposed = False    
    # decomposed deployment
    if(decomposed):
        for i in nfunctor_graph.action_runtime_table.data:
            if i.is_data():
                i.core_id = i.execution_id*2 + 1
            else:
                i.core_id = 48 - i.execution_id*2 - 1
        
    else:
        global num_execution_core
        num_control_core = num_execution_core
        num_data_core = 24 - num_execution_core

        print("deploying ")
        avaliable_cores_data = []
        avaliable_cores_control = []
        for i in range(num_data_core):
            avaliable_cores_data.append((i*2+1)%48)
        for i in range(num_execution_core):
            avaliable_cores_control.append((i*2 + num_data_core*2 + 1)%48)

        data_core_index = 0
        control_core_index = 0
        for i in nfunctor_graph.action_runtime_table.data:
            if i.is_data():
                i.core_id = avaliable_cores_data[i.execution_id % num_data_core]
            else:
                i.core_id = avaliable_cores_control[i.execution_id % num_control_core]
            #print(i.core_id, i.execution_id)

    
    # install action
    for i in nfunctor_graph.action_runtime_table.data:
        action_index = nfunctor_graph.action_runtime_table.data.index(i)
        if(i.action.action.name =="packet_in"):
            pass

        elif(i.action.action.name =="packet_out"):
            pass
        else:
            #print(i.action.action.registration_id,action_index,i.state_id, i.core_id)
            nflambda.send_command(command_action.with_argument(
                ["create", str(i.action.action.registration_id), str(action_index),
                 str(i.state_id), str(i.core_id)]))
            pass
    
    # install action
    for i in nfunctor_graph.action_runtime_table.data:
        action_index = nfunctor_graph.action_runtime_table.data.index(i)
        if(i.action.action.name =="packet_in"):
            nflambda.send_command(
                command_config.with_argument(["rss_reta_config", str(i.starting_bucket), str(i.num_bucket), str(i.core_id), str(action_index)]))

            next_action = i.next_action
            nflambda.send_command(command_worker.with_argument(
                ["first_action", str(i.core_id), str(next_action), str(i.starting_bucket), str(i.num_bucket)]))

        elif(i.action.action.name =="packet_out"):
            #print(["create_packet_out", str(action_index), str(i.core_id)])
            # configure last action
            nflambda.send_command(command_action.with_argument(["create_packet_out", str(action_index), str(i.core_id)]))
            pass
        else:
            #print(i.action.action.registration_id,action_index,i.state_id, i.core_id)
            nflambda.send_command(command_action.with_argument(
                ["create", str(i.action.action.registration_id), str(action_index),
                 str(i.state_id), str(i.core_id)]))
            pass
    
    # print(action_id,action_index,state_index,)
    
    # install path
    for i in nfunctor_graph.state_runtime_table.data:
        state_id = nfunctor_graph.state_runtime_table.data.index(i)

        nflambda.send_command(
                command_state.with_argument(["create", str(0), str(state_id)]))

        for j in i.path:
            if j!=-1:
                path_index = i.path.index(j)
                path_id = j
                # print(path_index,path_id,state_id)        
                nflambda.send_command(command_state.with_argument(["install_simple_path", str(state_id), str(j), str(path_index)]))
                
                

    sleep(1)    
    nflambda.run()
    

    
    sleep(5)      

In [ ]:
# Get throughput
# return value * 1.8 = actual throuput (Gb/s)
def get_throughput():
    throughput_sum = 0
    while ((throughput_sum <=8) or (throughput_sum >=75)):
        throughput = []
        get_flag = 0
        while(get_flag == 0):
            try:
                temp = get_nflambda_log()
                get_flag = 1
            except:
                print("trying restart...")
                restart_nf()
                update_deployment()
                sleep(10)

        #throughput.append(temp['worker']['1_throughput'][-1])
        index = 1
        throughput_sum = 0
        while index <= 47:
            throughput.append(temp['worker'][str(index)+'_throughput'][-1])
            throughput_sum += temp['worker'][str(index)+'_throughput'][-1]
            index += 2
    return throughput_sum

In [ ]:
# Initialize RL state

def state_initialize():
    global num_execution_core
    num_execution_core = 6
    update_deployment()
    return update_observation()

def state_initialize_training():
    global num_execution_core
    num_execution_core = 6
    return get_counter_training()

In [ ]:
# Sweep test
num_execution_core = 4
performance_sweep = {}
observation_sweep = {}
for core_n in range(17):
    print("current control core:", num_execution_core)
    throughput_avg = 0
    sleep(1)
    update_deployment()
    sleep(2)
    throughput_avg = get_throughput()
    sleep(1)
    state_new = update_observation()
    observation_sweep[num_execution_core] = state_new
    performance_sweep[num_execution_core] = throughput_avg
    num_execution_core += 1


In [ ]:
# For faster training, record the results, and generate random noise

import random
def get_performance_training():
    core = num_execution_core
    performance_obs = performance_sweep[core] * random.uniform(0.98,1.02)
    return performance_obs
    
def get_counter_training():
    core = num_execution_core
    counter = observation_sweep[core]
    counter_obs = []
    for row in counter[:-1]:
        row_obs = []
        for value in row:
            row_obs.append(value * random.uniform(0.95,1.05))
        counter_obs.append(row_obs)
    counter_obs.append(counter[-1])
    return counter_obs
        

In [ ]:
# Parameters

In [ ]:
OBSERVATION_SPACESHAPE = len(hardware_counters)*len(hardware_counters[0]) # only contains cpu_counters
MAXIMUM_UPDATE_NUM = 12 # larger value results as easier transfer onto a new environment (and better performance once trained), smaller value make faster convergence  
IMMEDIATE_UPDATE_THRESHOLD = 1.1 # immediately update the agent when an action generates obviously better performance
IMMEDIATE_ROLLBACK_THRESHOLD = 0.9 # immediately rollback an action if it results as a significantly worse performance
GAIN_AMP = 32 # amplifier to give non-linear rewards for actions
GAIN_EMP = 128 # amplifier to give more significant rewards for outstanding good actions.
UPDATE_LIMIT = 1.05

In [ ]:
num_iterations = 128
collect_episodes_per_iteration = MAXIMUM_UPDATE_NUM 
replay_buffer_capacity = 8192 
fc_layer_params = (128,)
learning_rate = 1e-3 
log_interval = 2 
num_eval_episodes = 2
eval_interval = 1 

In [ ]:
logaction = open("logged_actions_10.txt",'a')

class NFUpdateEnv_BIN(py_environment.PyEnvironment):

  def __init__(self):

    # Action: possible updates
    # Simpified as a binary operation, 
    # 0 relocates a data core to control, 1 relocates a control core to data
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, maximum=1, name='action')
    
    # Observation: normalized hardware performance counters
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(len(hardware_counters),len(hardware_counters[0])), dtype=np.float32, minimum=0, maximum=1, name='observation')
    
    # Train with random noise
    self._state = state_initialize_training()
    # Train with real task
    self._state = state_initialize()

    
    self._search_ended = False
    self._num_ite = 0
    self._search_ended_abn = 0
    self.best_result = 1
    self.ite_optimal = 0
    self.ite_best = 0
    self._step_counter = 0
    self._gain_record = 0
    self._hm_record = []
    self._good_search = 0
    self._new_record = 0    
    
    
  def step_counter(self):
    return self._step_counter

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    print("RESET...")
    # Train with random noise
    self._state = state_initialize_training()
    # Train with real task
    self._state = state_initialize()

    self._search_ended = False
    self._num_ite = 0
    self.ite_best += 1
    self.ite_optimal += 1
    self._search_ended_abn = 0
    self._gain_record = []
    self._good_search = 0
    self._new_record = 0
    return ts.restart(np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32))
    logaction.write("RESET\n")
    logaction.flush()
    
  def _reset_op(self):
    # Train with random noise
    self._state = state_initialize_training()
    # Train with real task
    self._state = state_initialize()

    self._search_ended = False
    #self._num_ite = 0
    return ts.termination(np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward=(4))


  def _step(self, action):
    print("ACTION GENERATED:",action)
    #print(self._num_ite)
    gain_amp = 0
    self._step_counter += 1

    if self._search_ended:
      print("SEARCH ENDED",self._state)
      #wait = input()
      logsearch = open("logged_search_10.txt",'a')
      logsearch.write("STEP COUNTER: " + str(self._step_counter) + "  >  GAIN RECORD: " + str(self._gain_record) + "\n")
      logsearch.flush()
      return self.reset()

    

    if self._num_ite >= MAXIMUM_UPDATE_NUM:
        if self._new_record <= self._throughput_initial :
              print("SEARCH ENDED WITH MAXIMUM TRY, NEGATIVE REWARD: ", -1)
              self._search_ended_abn = 2
              self._search_ended = True
        else:
              print("SEARCH ENDED WITH MAXIMUM TRY, POSITIVE REWARD: ", 1)
              self._search_ended_abn = 4
              self._search_ended = True            
      #return ts.termination(np.array(self._state, dtype=np.float32), reward=(-8))


    if self._num_ite == 0:
        # Train with random noise
        self._throughput_initial = get_performance_training()
        # Train with real task
        self._throughput_initial = get_throughput()
        
        self._throughput_old = self._throughput_initial
        
    update_flag = update_current_execution_state(action)
    
    if update_flag != -1:
        #update_deployment()
        # Train with random noise
        state_new = get_counter_training()
        # Train with real task
        state_new = update_observation()

        # Train with random noise
        throughput_new = get_performance_training()
        # Train with real task        
        # Wait until Stable
        sleep(1)
        throughput_new = get_throughput()
        
        while ((throughput_new >= 75) or (throughput_new <= 10)):
                    # Train with random noise
                    throughput_new = get_performance_training()
                    # Train with real task        
                    # Wait until Stable
                    sleep(1)
                    throughput_new = get_throughput()
                    print("bad value, re-test...")
        if throughput_new >= self._throughput_initial * 1.02:
            self._good_search = 1
        self._state = state_new
        gain = throughput_new / self._throughput_old
        print("ACTION TAKE:", action)
        print("PERFORMANCE UPDATE:", self._throughput_old, "-->", throughput_new)
        #print("STATE:", self._state)
        self._throughput_old = throughput_new
        logaction.write("ACTION:"+str(action)+" - NUM_CONTROL:"+str(num_execution_core) + " - THROUGHPUT:" + str(throughput_new) + "\n")
        logaction.flush()
        self._new_record = throughput_new

    else:
        gain = 1.0
        throughput_new = self._throughput_old
        print("ACTION NOT TAKE (INVALID):", action)
        logaction.write("ACTION:"+str(action)+" - NUM_CONTROL:"+str(num_execution_core) + " - INVALID" + "\n")
        logaction.flush()
    #gain_o = gain_o[0]
    #gain_n = gain_n[0]

    self._gain_record.append(throughput_new)
    if gain >= 1.02:
      #self._state = state_update(self._state, action)
      if throughput_new > self.best_result * UPDATE_LIMIT: 
        self.best_result = throughput_new
        print("NEW BEST", self.best_result, "IN ITE", self._step_counter)
        logbest = open("logged_best_10.txt",'a')
        logbest.write(str(self.best_result) + " > " + str(self._state) + " > " + str(self._step_counter) + "\n")
        logbest.flush()
        print("SEARCH ENDED WITH OPTIMAL PLAN")
        #wait = input()
        print("NEW OPT", throughput_new, "IN ITE", self.ite_optimal)
        self.ite_best = 0
        self.ite_optimal = 0
        self._search_ended = True
    #else:
    #  self._state = state_update(self._state, action)
    
    
    self._num_ite += 1
    self._observation_spec = self._state[:OBSERVATION_SPACESHAPE]

    # Reinforcement Gain
    RGAIN = 16

    if self._search_ended:
        if self._search_ended_abn == 1:
              return ts.termination(np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward=(-1))
        if self._search_ended_abn == 2:
              return ts.termination(np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward=(-1)) 
        if self._search_ended_abn == 4:
              return ts.termination(np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward=(1)) 
        if self._search_ended_abn == 0:
              gain = throughput_new / self._throughput_initial
              reward = (((gain)**2)) * RGAIN
              print("ENHANCED POSITIVE REWARD:", reward)
              return ts.termination(np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward)
    else:
      if gain > 1:
          print("POSITIVE REWARD:",(((gain)**2)-1)*10)
          return ts.transition(
              np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward=(((gain)**2-1)*10), discount=0.9)
      elif gain == 1.0:
          print("INVALID ACTION, -1 REWARD")
          return ts.transition(
              np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward=-1, discount=1.0)          
      else:
          print("NEGATIVE REWARD:",(((1/gain)**2)-1)*10)
          #return ts.transition(
          #    np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward=-(64**(1/gain)/64)+1, discount=1.0)
          return ts.transition(
              np.array(self._state[:OBSERVATION_SPACESHAPE], dtype=np.float32), reward=-(((1/gain)**2-1)*10), discount=0.9)


In [ ]:
env = NFUpdateEnv_BIN()
global_step = tf.compat.v1.train.get_or_create_global_step()
action = np.array([1], dtype=np.int32)


In [ ]:
train_py_env = env
eval_py_env = env

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

actor_net = actor_distribution_network.ActorDistributionNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

tf_agent = reinforce_agent.ReinforceAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    actor_network=actor_net,
    optimizer=optimizer,
    normalize_returns=True,
    train_step_counter=train_step_counter)
#tf_agent.initialize()

policy = tf_agent.policy
eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

policy_checkpointer = common.Checkpointer(ckpt_dir=os.path.join("./ckpts/", 'checkpoint_test'),
                                                policy=policy)
policy_checkpointer.initialize_or_restore()

eval_policy = tf_agent.policy
collect_policy = tf_agent.collect_policy

def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      tf_agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
      replay_buffer_signature)
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    table_name=table_name,
    sequence_length=None,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddEpisodeObserver(
    replay_buffer.py_client,
    table_name,
    replay_buffer_capacity
)

def collect_episode(environment, policy, num_episodes):

  driver = py_driver.PyDriver(
    environment,
    py_tf_eager_policy.PyTFEagerPolicy(
      policy, use_tf_function=True),
    [rb_observer],
    max_episodes=num_episodes)
  initial_time_step = environment.reset()
  driver.run(initial_time_step)

print("Loaded weights")
print(actor_net.trainable_weights[0])

In [ ]:
tf_agent.train = common.function(tf_agent.train)

# Reset the train step
tf_agent.train_step_counter.assign(0)

avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  collect_episode(
      train_py_env, tf_agent.collect_policy, collect_episodes_per_iteration)

iterator = iter(replay_buffer.as_dataset(sample_batch_size=1))
  trajectories, _ = next(iterator)
  #print(iterator)
  #print(trajectories)
  train_loss = tf_agent.train(experience=trajectories)  

  replay_buffer.clear()
  #wait = input()
  step = tf_agent.train_step_counter.numpy()
  print(">>>>>>>>>>>>>", step)

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, tf_agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    restart_flag = 0
    while (restart_flag == 0):
      try:
        restart_nf()
        restart_flag = 1
      except:
        restart_flag = 0

    checkpoint_dir = os.path.join("./ckpts/", 'checkpoint_test_re10')

    train_checkpointer = common.Checkpointer(
        ckpt_dir=checkpoint_dir,
        max_to_keep=1,
        agent=tf_agent,
        policy=tf_agent.policy,
        replay_buffer=replay_buffer,
        global_step=global_step
    )

    tf_policy_saver = policy_saver.PolicySaver(tf_agent.policy)
    policy_dir = os.path.join("./ckpts/", 'checkpoint_test_re10')
    tf_policy_saver.save(policy_dir)
    train_checkpointer.save(global_step)
    saved_policy = tf.saved_model.load(policy_dir)
    returns.append(avg_return)